# Multi Resolution Simulation

### Classes and modules

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from IPython.display import display
import copy

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

GPU Ocean-modules:

In [2]:
from gpuocean.utils import IPythonMagic, Common, NetCDFInitialization

In [3]:
%cuda_context_handler gpu_ctx

In [4]:
%cuda_context_handler gpu_ctx_refined

## Selecting loc

As initial and boundary conditions to the simulation, we use data from the Norkyst800 s-level files:

In [5]:
nk800_url  = ["/sintef/data/NorKyst800/ocean_his.an.20190716.nc"]

data_args = NetCDFInitialization.removeMetadata(NetCDFInitialization.getInitialConditionsNorKystCases(nk800_url, "lofoten", download_data=False, norkyst_data=False))
data_args.keys()

dict_keys(['nx', 'ny', 'dx', 'dy', 'g', 'r', 'H', 'eta0', 'hu0', 'hv0', 'angle', 'latitude', 'f', 'boundary_conditions_data', 'boundary_conditions', 'wind'])

In [6]:
sim_args = {
    "dt": 0.0,
    "write_netcdf":False,
    "model_time_step": 60
     }

## Simulation

In [7]:
from gpuocean.SWEsimulators import CDKLM16

from importlib import reload
reload(CDKLM16)

<module 'gpuocean.SWEsimulators.CDKLM16' from '/home/florianb/havvarsel/gpuocean/src/gpuocean/SWEsimulators/CDKLM16.py'>

In [8]:
sim = CDKLM16.CDKLM16(gpu_ctx, **sim_args, **data_args)
sim.give_birth(gpu_ctx_refined, [[100, 100], [200,300]], 1.2)

This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!
This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!


Use halo mask according to bathymetry
Construct halo mask according to finer bathymetry information!


In [9]:
cdklm_swe_2D = sim.kernel.get_function("cdklm_swe_2D")
cdklm_swe_2D.prepare("fiPiPiPiPiPiPiPiPifffiPi")

In [10]:
fluxes_N = Common.CUDAArray2D(sim.gpu_stream, sim.nx, 3, 2, 0, np.zeros((3,sim.nx+4)))
# Somehow the allocation is necessary to avoid storage issues?!?!
fill = Common.CUDAArray2D(sim.gpu_stream, sim.nx, 300, 2, 0, np.zeros((300,sim.nx+4)))

In [11]:
cdklm_swe_2D.prepared_async_call(sim.global_size, sim.local_size, sim.gpu_stream, \
                           np.float32(0.1), \
                           np.int32(1), \
                           sim.gpu_data.h0.data.gpudata, sim.gpu_data.h0.pitch, \
                           sim.gpu_data.hu0.data.gpudata, sim.gpu_data.hu0.pitch, \
                           sim.gpu_data.hv0.data.gpudata, sim.gpu_data.hv0.pitch, \
                           sim.gpu_data.h1.data.gpudata, sim.gpu_data.h1.pitch, \
                           sim.gpu_data.hu1.data.gpudata, sim.gpu_data.hu1.pitch, \
                           sim.gpu_data.hv1.data.gpudata, sim.gpu_data.hv1.pitch, \
                           sim.bathymetry.Bi.data.gpudata, sim.bathymetry.Bi.pitch, \
                           sim.bathymetry.Bm.data.gpudata, sim.bathymetry.Bm.pitch, \
                           sim.bathymetry.mask_value,
                           np.float32(0), \
                           np.float32(0), \
                           np.int32(0),
                           fluxes_N.data.gpudata, fluxes_N.pitch)

In [12]:
fluxes_N.download(sim.gpu_stream)

array([[  0.,   0.,   2., ..., 497.,   0.,   0.],
       [  0.,   0.,   2., ..., 497.,   0.,   0.],
       [  0.,   0.,   2., ..., 497.,   0.,   0.]], dtype=float32)